In [ ]:
# import findspark
# findspark.init()
# from pyspark.sql import SparkSession
# spark = SparkSession.builder.appName("stock_exchange").getOrCreate()

In [ ]:
# pip install yfinance

In [ ]:
# pip install pandas_datareader

In [ ]:
# pip install --upgrade kiteconnect

In [ ]:
# pip install -U pyopenssl

In [ ]:
# pip install --upgrade attrs

In [ ]:
# pip install kafka-python

In [ ]:
# import logging
# from kiteconnect import KiteConnect


# logging.basicConfig(level=logging.DEBUG)

In [ ]:

# kite = KiteConnect(api_key="your_api_key")

# Redirect the user to the login url obtained
# from kite.login_url(), and receive the request_token
# from the registered redirect url after the login flow.
# Once you have the request_token, obtain the access_token
# as follows.


In [ ]:
# data = kite.generate_session("request_token_here", api_secret="your_secret")
# kite.set_access_token(data["access_token"])


In [ ]:
# # Place an order
# try:
#     order_id = kite.place_order(tradingsymbol="INFY",
#                                 exchange=kite.EXCHANGE_NSE,
#                                 transaction_type=kite.TRANSACTION_TYPE_BUY,
#                                 quantity=1,
#                                 variety=kite.VARIETY_AMO,
#                                 order_type=kite.ORDER_TYPE_MARKET,
#                                 product=kite.PRODUCT_CNC,
#                                 validity=kite.VALIDITY_DAY)

#     logging.info("Order placed. ID is: {}".format(order_id))
# except Exception as e:
#     logging.info("Order placement failed: {}".format(e.message))

# # Fetch all orders
# kite.orders()

# # Get instruments
# kite.instruments()

# # Place an mutual fund order
# kite.place_mf_order(
#     tradingsymbol="INF090I01239",
#     transaction_type=kite.TRANSACTION_TYPE_BUY,
#     amount=5000,
#     tag="mytag"
# )

# # Cancel a mutual fund order
# kite.cancel_mf_order(order_id="order_id")

# # Get mutual fund instruments
# kite.mf_instruments()

In [ ]:
# For reading stock data from yahoo
from pandas_datareader.data import DataReader
import yfinance as yf
from pandas_datareader import data as pdr
from datetime import datetime
import pandas as pd
import matplotlib.pyplot as plt


In [ ]:
# The tech stocks we'll use for this analysis
tech_list = ['AAPL']
# Set up End and Start times for data grab
tech_list = ['AAPL']

end = datetime.now()
start = datetime(end.year - 1, end.month, end.day)

for stock in tech_list:
    globals()[stock] = yf.download(stock, start, end)
    

company_list = [AAPL]
company_name = ["APPLE"]

for company, com_name in zip(company_list, company_name):
    company["company_name"] = com_name
    
df = pd.concat(company_list, axis=0)
df.tail(10)

In [ ]:
AAPL

In [ ]:
# Let's see a historical view of the closing price
plt.figure(figsize=(15, 10))
plt.subplots_adjust(top=1.25, bottom=1.2)

for i, company in enumerate(company_list, 1):
    plt.subplot(2, 2, i)
    company['Adj Close'].plot()
    plt.ylabel('Adj Close')
    plt.xlabel(None)
    plt.title(f"Closing Price of {tech_list[i - 1]}")
    
plt.tight_layout()

In [ ]:
df_sample = yf.download("AMZN AAPL", start, end)
df_sample.head()


In [ ]:
yf.download(tickers='INFY', period='1d', interval='1m')

In [ ]:
# buyer and seller
# assurance of stock trade
# instrument is just the stock
# multiple orders can get collected

## generate all of these orders - 4 instruments - buyer sellers random data - write a Producer data

## Keep track of buy and sell orders - match buy to a sell
# this will keep on happening - once buy and sell matches it will result in a trade

## Keep track of trades and take 10minute trade windows for profit and SMA calc

# Output can be shown on console, write to file or anything is fine - meaningful format


In [ ]:
# importing the required modules  
import json
from json import loads
from time import sleep
from json import dumps
from kafka import KafkaProducer


In [ ]:
from faker import Faker
import random
from faker.providers import BaseProvider
from datetime import datetime
import time

In [ ]:
# Kafka producer configuration
producer = KafkaProducer(bootstrap_servers=['0.0.0.0:9092'],
                         value_serializer=lambda m: json.dumps(m).encode('ascii'))

# producer.send("test-topic",
#     value={"message": "hello world"}
# )
# producer.flush()


In [ ]:
fake = Faker()

# Create required fields with values
class InstrumentProvider(BaseProvider):
    def instr_name(self):
        instr_names = ["RIL", "INFY", "ONGC", "TCS"]
        return instr_names[random.randint(0,len(instr_names)-1)]
    
    def trxn_type(self):
        trxn_type = ["BUY", "SELL"]
        return trxn_type[random.randint(0,len(trxn_type)-1)]
    
    def exchange(self):
        trxn_type = ["NSE", "BSE"]
        return trxn_type[random.randint(0,len(trxn_type)-1)]

fake.add_provider(InstrumentProvider)

In [ ]:
# creating function to generate the Order
def produce_order(orderid=1):
    instrument = fake.instr_name()
    trxn_type = fake.trxn_type()
    quantity = round(random.uniform(2, 5), 0)
    price = round(random.uniform(10, 20), 0)
    order_ts = datetime.now().strftime("%Y/%m/%d, %H:%M:%S")
    
    # message
    message = {
        'id': orderid+1000000,
        'instrument': instrument,
        'trxn_type': trxn_type,
        'quantity': quantity,
        'price': price,
        'order_timestamp': order_ts,
    }
    return message



In [ ]:
# produce_order(8)

In [ ]:
# Produce buy/sell orders
i=1
while i < 1000:
    message = produce_order(i)

    print("Sending: {}".format(message))
    # sending the message to Kafka
    producer.send("stock_exchange_orders",
                  value=message)
    # 1 seconds of sleep time before the next message
    time.sleep(0.5)

    # Force sending of all messages
    if (i % 100) == 0:
        producer.flush()
    i=i+1
producer.flush()